EDA and Preprocessing

In [47]:
#importing libraries
import numpy as np
import pandas as pd

In [48]:
any_2015 = pd.read_csv("2015.csv")
any_2016 = pd.read_csv("2016.csv")
any_2017 = pd.read_csv("2017.csv")
any_2018 = pd.read_csv("2018.csv")
any_2019 = pd.read_csv("2019.csv")

In [49]:
any_2017 = any_2017.rename(columns = {
    'Happiness.Rank':'Happiness Rank', 'Economy..GDP.per.Capita.':'GDP per Capita', 
    'Health..Life.Expectancy.':'Health (Life Expectancy)','Trust..Government.Corruption.':'Trust (Government Corruption)'
})

any_2018 = any_2018.rename(columns = {
    'Country or region':'Country', 'Overall rank':'Happiness Rank', 'GDP per capita':'GDP per Capita',
    'Social support':'Family', 'Healthy life expectancy':'Health (Life Expectancy)', 'Freedom to make life choices':'Freedom',
    'Perceptions of corruption':'Trust (Government Corruption)'
})
any_2019 = any_2019.rename(columns = {
    'Country or region':'Country', 'Overall rank':'Happiness Rank', 'GDP per capita':'GDP per Capita',
    'Social support':'Family', 'Healthy life expectancy':'Health (Life Expectancy)', 'Freedom to make life choices':'Freedom',
    'Perceptions of corruption':'Trust (Government Corruption)'
})

In [50]:
dfs = [any_2015, any_2016, any_2017, any_2018, any_2019]

In [51]:
df = pd.concat(dfs) #unim datasets
df =df.sort_values(by = ['Country']) #els ordenem per país

print("Número de mostres: ", len(df), "\n")

for col in df.columns:
    print(col)
    print("nulls: ", df[col].isnull().sum(), "\n")

Número de mostres:  782 

Country
nulls:  0 

Region
nulls:  467 

Happiness Rank
nulls:  0 

Happiness Score
nulls:  467 

Standard Error
nulls:  624 

Economy (GDP per Capita)
nulls:  467 

Family
nulls:  0 

Health (Life Expectancy)
nulls:  0 

Freedom
nulls:  0 

Trust (Government Corruption)
nulls:  1 

Generosity
nulls:  0 

Dystopia Residual
nulls:  467 

Lower Confidence Interval
nulls:  625 

Upper Confidence Interval
nulls:  625 

Happiness.Score
nulls:  627 

Whisker.high
nulls:  627 

Whisker.low
nulls:  627 

GDP per Capita
nulls:  315 

Dystopia.Residual
nulls:  627 

Score
nulls:  470 



In [52]:
df = df.drop(columns=['Happiness Score','Standard Error','Dystopia Residual','Lower Confidence Interval','Upper Confidence Interval','Happiness.Score','Whisker.high','Whisker.low','Dystopia.Residual','Score',])
#eliminem les columnes amb 467 o més valors nulls, menys GDP per càpita
df = df.reset_index()
df = df.drop(index=df.iloc[np.where(df['Country'] == 'Northern Cyprus')].index)
df = df.drop(index=df.iloc[np.where(df['Country'] == 'Northern Macedonia')].index)

df.at[625,['Country']] = 'Somaliland Region'

Tractament dels valors nuls restants

In [53]:
countries = list(set(df['Country'].values))
countries = sorted(countries)

In [54]:
new_regions = []
for i in range(len(countries)):
        country_index = np.where(df['Country'] == countries[i]) # Get indexes of current country
        current_country_df = df.iloc[country_index] #get values of current indexes
        if all(pd.isnull(current_country_df['Region'].values)): # if all null for regions, drop country from df
            country_index = np.where(df['Country'] == countries[i])[0]
            for vals in country_index:
                try:
                    df = df.drop(index = vals)
                except KeyError:
                    pass
        else:
            region_name_pos = np.where([type(x) == str for x in current_country_df['Region'].values])[0][0]
            region = current_country_df['Region'].values[region_name_pos]
            current_country_df = current_country_df.replace(np.nan, region)
            new_regions.append(current_country_df['Region'].values)

In [55]:
new_regions = list(new_regions)
k = 0
region_vals = []
for arr in new_regions:
    for vals in arr:
        region_vals.append(vals)
        k+=1
        
len(region_vals)

771

In [58]:
df['Region'] = region_vals
df.head(-1)

,index,Country,Region,Happiness Rank,Economy (GDP per Capita),Family,Health (Life Expectancy),Freedom,Trust (Government Corruption),Generosity,GDP per Capita
0,153,Afghanistan,Southern Asia,154,0.38227,0.110370,0.173440,0.16430,0.071120,0.312680,NaN
1,152,Afghanistan,Southern Asia,153,0.31982,0.302850,0.303350,0.23414,0.097190,0.365100,NaN
2,144,Afghanistan,Southern Asia,145,NaN,0.537000,0.255000,0.08500,0.036000,0.191000,0.332000
3,153,Afghanistan,Southern Asia,154,NaN,0.517000,0.361000,0.00000,0.025000,0.158000,0.350000
4,140,Afghanistan,Southern Asia,141,NaN,0.581543,0.180747,0.10618,0.061158,0.311871,0.401477
...,...,...,...,...,...,...,...,...,...,...,...
776,137,Zambia,Sub-Saharan Africa,138,NaN,1.058000,0.426000,0.43100,0.087000,0.247000,0.578000
777,143,Zimbabwe,Sub-Saharan Africa,144,NaN,1.094000,0.248000,0.40600,0.099000,0.132000,0.357000
778,145,Zimbabwe,Sub-Saharan Africa,146,NaN,1.114000,0.433000,0.36100,0.089000,0.151000,0.366000
779,114,Zimbabwe,Sub-Saharan Africa,115,0.27100,1.032760,0.334750,0.25861,0.080790,0.189870,NaN


In [59]:
#Convertire les regions en variables categóriques i les afegiré com a columnes ja que poden
#aportar informació interessant

In [60]:
one_hot_regions = pd.get_dummies(df['Region'],prefix = 'Region') #convertim les regions en un dataset categóric
df = df.join(one_hot_regions) #afegim les noves columnes al dataframe principal
df = df.drop(columns = 'Region') #eliminem la columna de les regions

In [61]:
#ara el data frame llueix així:
df

,index,Country,Happiness Rank,Economy (GDP per Capita),Family,Health (Life Expectancy),Freedom,Trust (Government Corruption),Generosity,GDP per Capita,Region_Australia and New Zealand,Region_Central and Eastern Europe,Region_Eastern Asia,Region_Latin America and Caribbean,Region_Middle East and Northern Africa,Region_North America,Region_Southeastern Asia,Region_Southern Asia,Region_Sub-Saharan Africa,Region_Western Europe
0,153,Afghanistan,154,0.38227,0.110370,0.173440,0.164300,0.071120,0.312680,NaN,0,0,0,0,0,0,0,1,0,0
1,152,Afghanistan,153,0.31982,0.302850,0.303350,0.234140,0.097190,0.365100,NaN,0,0,0,0,0,0,0,1,0,0
2,144,Afghanistan,145,NaN,0.537000,0.255000,0.085000,0.036000,0.191000,0.332000,0,0,0,0,0,0,0,1,0,0
3,153,Afghanistan,154,NaN,0.517000,0.361000,0.000000,0.025000,0.158000,0.350000,0,0,0,0,0,0,0,1,0,0
4,140,Afghanistan,141,NaN,0.581543,0.180747,0.106180,0.061158,0.311871,0.401477,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
777,143,Zimbabwe,144,NaN,1.094000,0.248000,0.406000,0.099000,0.132000,0.357000,0,0,0,0,0,0,0,0,1,0
778,145,Zimbabwe,146,NaN,1.114000,0.433000,0.361000,0.089000,0.151000,0.366000,0,0,0,0,0,0,0,0,1,0
779,114,Zimbabwe,115,0.27100,1.032760,0.334750,0.258610,0.080790,0.189870,NaN,0,0,0,0,0,0,0,0,1,0
780,130,Zimbabwe,131,0.35041,0.714780,0.159500,0.254290,0.085820,0.185030,NaN,0,0,0,0,0,0,0,0,1,0
